In [55]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import sklearn as sk
dataset = load_boston()

c:\Users\bitcamp\anaconda3\envs\tf274gpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California ho

In [56]:
x = dataset.data
y = dataset.target
x= pd.DataFrame(x)
# print("x 최솟값 : ",np.min(x))
# print("x 최댓값 : ",np.max(x))

In [57]:
# 스케일링 하기
scaler = StandardScaler()
# 가중치 생성후
scaler.fit(x)
# 변환
x = scaler.transform(x)
print("x 최솟값 : ",np.min(x))
print("x 최댓값 : ",np.max(x))

x 최솟값 :  -3.9071933049810337
x 최댓값 :  9.933930601860268


In [58]:
# 3. 모델구성
# 실습 train 0.7 이상
# 평가지표 R2 : 0.8 이상으로 뽑아내보기 / RMSE 사용

x_train, x_test,y_train,y_test = train_test_split(x,y,
    train_size=0.7,
    shuffle = True,
    random_state= 123
)

print(len(x_train))
print(len(x_test))

354
152


In [59]:
# 13개의 칼럼 을 가지고있는 데이터를 조회합니다
print(x.shape) #(506,13)
# print(x)
print(y.shape) #(506,)
# print(y)


(506, 13)
(506,)


In [60]:
# 사이킥 런의 칼럼 조회기능
print(dataset.feature_names)
print(len(dataset.feature_names))


['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
13


In [61]:
# 사이킥런의 데이터 요약
# print(dataset.DESCR)

In [62]:
# 3. 모델구성
# 실습 train 0.7 이상
# 평가지표 R2 : 0.8 이상으로 뽑아내보기 / RMSE 사용

x_train, x_test,y_train,y_test = train_test_split(x,y,
    train_size=0.7,
    shuffle = True,
    random_state= 123
)

scaler = MinMaxScaler()
# 가중치 생성후
scaler.fit(x_train)
# x_train 의 가중치 변환
x_train = scaler.transform(x_train)
# x_test 는 변환만 !
x_test = scaler.transform(x_test)

print(len(x_train))
print(len(x_test))

354
152


In [63]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Input,Dropout
# path = "../_save/"
# model = load_model(path+"MCP/keras30_ModelCheckPoint1.hdf5")

# model = Sequential([
#     Dense(50,input_dim=13,activation="relu"),
#     Dropout(0.5),
#     Dense(50,activation="relu"),
#     Dense(50,activation="relu"),
#     Dense(50,activation="relu"),
#     Dense(50,activation="relu"),
#     Dense(1)
# ])


# 3. 모델구성 함수형
input1 = Input(shape=(13,))
dense1 = Dense(32,activation="relu")(input1)
drop1 = Dropout(0.5)(dense1)
dense3 = Dense(128,activation="relu")(drop1)
dense4 = Dense(64,activation="relu")(dense3)
dense5 = Dense(32,activation="relu")(dense4)
output1 = Dense(1,activation="relu")(dense5)
model=Model(inputs=input1,outputs=output1)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense_35 (Dense)            (None, 32)                448       
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_36 (Dense)            (None, 128)               4224      
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 32)                2080      
                                                                 
 dense_39 (Dense)            (None, 1)                 33  

In [64]:
# 모델저장

import datetime
date = datetime.datetime.now()

# string for time 문자열로 바꿔
date =date.strftime("%y년 %m월 %d일 %H시 %m분")
print(type(date))
print(date)

path = "../_save/"
filepath = "../_save/MCP/"

# 에폭과 val_loss 소수4째자리까지
filename = '{epoch:04d}-{val_loss:.4f}.hdf5'        

# filename="{epoch:.04d}-{val_loss:4f}.hdf5"

<class 'str'>
23년 01월 12일 16시 01분


In [65]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


earlyStopping = EarlyStopping(monitor="val_loss",
                              mode="min",patience =10,
                              # True 면 좋았던 웨이트로 복원됨
                              # False 면 마지막 웨이트로 놔둠..
                              restore_best_weights=True,
                              verbose=1                              
                              )
# 모델체크포인트는 가장 낮은 val_loss 가 갱신 될 떄 마다 저장합니다.

mcp = ModelCheckpoint(
    monitor="val_loss",
    mode="auto",
    save_best_only=True,
    filepath= (filepath+'k31_'+date+filename),
    verbose=1 
)
                    
    
model.compile(loss="mse",optimizer="adam")

# model.fit 이 반환한 기록을 저장
hist = model.fit(
                x_train,
                y_train,
                epochs=100,
                batch_size=3,
                validation_split=0.2,
                verbose=1,
                callbacks = [earlyStopping,mcp],
                
                )




Epoch 1/100
88/95 [==========================>...] - ETA: 0s - loss: 344.2213
Epoch 00001: val_loss improved from inf to 96.79230, saving model to ../_save/MCP\k31_23년 01월 12일 16시 01분0001-96.7923.hdf5
95/95 [==============================] - 1s 5ms/step - loss: 330.3735 - val_loss: 96.7923
Epoch 2/100
84/95 [=========================>....] - ETA: 0s - loss: 97.0330 
Epoch 00002: val_loss improved from 96.79230 to 64.80330, saving model to ../_save/MCP\k31_23년 01월 12일 16시 01분0002-64.8033.hdf5
95/95 [==============================] - 0s 4ms/step - loss: 94.7201 - val_loss: 64.8033
Epoch 3/100
87/95 [==========================>...] - ETA: 0s - loss: 79.4648
Epoch 00003: val_loss improved from 64.80330 to 49.07772, saving model to ../_save/MCP\k31_23년 01월 12일 16시 01분0003-49.0777.hdf5
95/95 [==============================] - 0s 3ms/step - loss: 78.8031 - val_loss: 49.0777
Epoch 4/100
84/95 [=========================>....] - ETA: 0s - loss: 85.2253
Epoch 00004: val_loss improved from 49.0777

In [66]:
# 성능평가
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def RMSE(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
# 평가 손실률 가져오기
loss = model.evaluate(x_test,y_test)

y_predict =model.predict(x_test)
# print("=================")
# print(y_test)
# print(y_predict)
# print("=================")
print("그냥출력")
print("loss : ",loss)
print("RMSE : ",RMSE(y_test,y_predict))
print("R2   : ",r2_score(y_test,y_predict))

5/5 [==============================] - 0s 2ms/step - loss: 32.1923
그냥출력
loss :  32.192317962646484
RMSE :  5.67382763720448
R2   :  0.6017186674755686


In [67]:
# # 모델불러오기
# from tensorflow.keras.models import load_model

# path = "../_save/"

# # 가중치 불러오기는 모델이 저장 되어 있어야 가능함
# model.load_weights(path+"save_weight2.h5")

In [68]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense_35 (Dense)            (None, 32)                448       
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_36 (Dense)            (None, 128)               4224      
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 32)                2080      
                                                                 
 dense_39 (Dense)            (None, 1)                 33  

In [69]:
# 불러온 모델 성능평가
model2 = load_model(path+"MCP/keras30_ModelCheckPoint3.h5")

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def RMSE(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
# 평가 손실률 가져오기
loss = model2.evaluate(x_test,y_test)

y_predict =model2.predict(x_test)
# print("=================")
# print(y_test)
# print(y_predict)
# print("=================")
print("load_model 출력")
print("loss : ",loss)
print("RMSE : ",RMSE(y_test,y_predict))
print("R2   : ",r2_score(y_test,y_predict))

5/5 [==============================] - 0s 1ms/step - loss: 16.9394
load_model 출력
loss :  16.939428329467773
RMSE :  4.115753662782388
R2   :  0.7904264734919755
